# Code export using Dill

[Dill](https://github.com/uqfoundation/dill) is a good library that offers broad support for serializing code and therefore also a number of utilities for inspecting code. 

The default serialization format is binary, and so perhaps not ideal for this project as we would want to capture the source somewhere for both security and maintainability purposes. 

However, we can relatively easily write our own 'code dumper' that will print objects in a more accessible way.

In [1]:
import random
import inspect
import dill
import pandas as pd

In [2]:
def write_code(func):
    """
    This function will attempt to access the source and globals of an object and write the source for these.
    """
    for name, mod in dill.detect.globalvars(func).items():
        print(dill.source.importable(mod, alias=name))
    print(dill.source.getsource(func))

In [3]:
# Our test function
def my_number(min=10, max=50):
    return f"My number is {random.randint(min, max)}"


In [4]:
write_code(my_number)

import random

def my_number(min=10, max=50):
    return f"My number is {random.randint(min, max)}"



# Importing Dill on Jupyter-lite

There is a slight issue importing Dill on Jupyter-lite. Dill attempts to detect object types by opening `os.devnull` in different modes, but this does not work for `r+` (and possibly other modes):

``` shell
File /lib/python3.10/site-packages/dill/_dill.py:145
    143 FileType = get_file_type('rb', buffering=0)
    144 TextWrapperType = get_file_type('r', buffering=-1)
--> 145 BufferedRandomType = get_file_type('r+b', buffering=-1)
    146 BufferedReaderType = get_file_type('rb', buffering=-1)
    147 BufferedWriterType = get_file_type('wb', buffering=-1)

File /lib/python3.10/site-packages/dill/_dill.py:138, in get_file_type(*args, **kwargs)
    136 def get_file_type(*args, **kwargs):
    137     open = kwargs.pop("open", __builtin__.open)
--> 138     f = open(os.devnull, *args, **kwargs)
    139     t = type(f)
    140     f.close()
    
UnsupportedOperation: File or stream is not seekable.

```

If we decide to use dill for this project, then we should probably submit a proper PR to them that will allow us to work around this. In the meantime I have come up with a rather horrible monkey-patch:


In [5]:
from contextlib import contextmanager

@contextmanager
def custom_devnull():
    import builtins
    import io
    import tempfile
    import _pyio

    from pathlib import Path

    with tempfile.TemporaryDirectory() as dir:
        testfile = f"{dir}/notdevnull"
        Path(testfile).touch()
        old_open = _pyio.open
        
        def open_tempfile(_, *args, **kwargs):
            return old_open(testfile, *args, **kwargs)
        
        def new_open(*args, **kwargs):
            try:
                return old_open(*args, **kwargs)
            except io.UnsupportedOperation:
                return open_tempfile(*args, **kwargs)
        
        builtins.open = _pyio.open = new_open
        
        yield
        
        builtins.open = _pyio.open = old_open


In [6]:
with custom_devnull():
    import dill

In [7]:
dill._dill.BufferedRandomType

_io.BufferedRandom